In [ ]:
%pip install opendatasets

import opendatasets as od

# This will prompt you for your Kaggle username/API key once
od.download("https://www.kaggle.com/datasets/tawfikelmetwally/census-income-dataset")

In [2]:
import pandas as pd

# Adjust filename depending on what was downloaded
df = pd.read_csv("census-income-dataset/adult.csv")

# Display first rows
df.head()

,Age,Workclass,Final Weight,Education,EducationNum,Marital Status,Occupation,Relationship,Race,Gender,Capital Gain,capital loss,Hours per Week,Native Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [68]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Step 1: Load and rename dataset columns for compatibility
df = pd.read_csv("census-income-dataset/adult.csv")

df.rename(columns={
    "Age": "age",
    "Workclass": "workclass",
    "Final Weight": "final_weight",
    "Education": "education",
    "EducationNum": "education_num",
    "Marital Status": "marital_status",
    "Occupation": "occupation",
    "Relationship": "relationship",
    "Race": "race",
    "Gender": "gender",
    "Capital Gain": "capital_gain",
    "capital loss": "capital_loss",
    "Hours per Week": "hours_per_week",
    "Native Country": "native_country",
    "Income": "income"
}, inplace=True)

# Step 2: Convert labels to binary
df['income'] = df['income'].apply(lambda x: 1 if x.strip() == ">50K" else 0)

# Step 3: Split features and labels
X_data = df.drop('income', axis=1)
y_label = df['income']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=42)

# Step 4: Define feature columns
feature_columns = [
    tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Male", "Female"]),
    tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000),
    tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000),
    tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000),
    tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000),
    tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000),
    tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000),

    tf.feature_column.numeric_column("age"),
    tf.feature_column.numeric_column("education_num"),
    tf.feature_column.numeric_column("capital_gain"),
    tf.feature_column.numeric_column("capital_loss"),
    tf.feature_column.numeric_column("hours_per_week"),
    tf.feature_column.numeric_column("final_weight"),
]

# Wrap categorical columns with indicator_column for LinearClassifier
feat_columns = []

for fc in feature_columns:
    if hasattr(fc, 'vocabulary_list') or hasattr(fc, 'hash_bucket_size'):
        feat_columns.append(tf.feature_column.indicator_column(fc))
    else:
        feat_columns.append(fc)

# Step 5: Define input functions
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_train,
    y=y_train,
    batch_size=128,
    num_epochs=None,
    shuffle=True
)

test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    x=X_test,
    y=y_test,
    batch_size=128,
    num_epochs=1,
    shuffle=False
)

# Step 6: Train the model
model = tf.estimator.LinearClassifier(feature_columns=feat_columns)
model.train(input_fn=train_input_fn, steps=5000)

# Step 7: Evaluate the model
evaluation = model.evaluate(input_fn=test_input_fn)
print("\nModel Evaluation:")
for key, value in evaluation.items():
    print(f"{key}: {value:.4f}")


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\ASUS\\AppData\\Local\\Temp\\tmpvtsg3l1i', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions 